# Zoomcamp MLOps Course Cohort 2024 Week 01 Homework
*Version:* `1.2` *(Jupytext, time measurements, logger, param notebook execution, fixes)*

<a name="ToC"></a>
# Table of Content

- [Notebook Description](#0)
- [General Settings](#1)
    - [Paths](#1-1)
    - [Notebook Functionality and Appearance](#1-2)
    - [External Libraries](#1-3)
    - [Internal Code](#1-4)
    - [Constants](#1-5)   
- [Analysis](#2)   
    - [Data Reading](#2-1)   
    - [Questions](#2-2)
        - [Q1 Read the data for January. How many columns are there?](#2-2-1)     
        - [Q2 Computing duration](#2-2-2)
        - [](#2-2-3)
        - [](#2-2-4)
        - [](#2-2-5)
        - [](#2-2-6)
- [Final Timestamp](#3)  

<a name="0"></a>
# Notebook Description
[ToC](#ToC) 

> *Please put your comments about the notebook functionality here.*  

<a name="1"></a>
# GENERAL SETTINGS
[ToC](#ToC)  
General settings for the notebook (paths, python libraries, own code, notebook constants). 

> *NOTE: All imports and constants for the notebook settings shoud be here. Nothing should be imported in the analysis section.*

<a name="1-1"></a>
### Paths
[ToC](#ToC)  

Adding paths that are necessary to import code from within the repository.

In [1]:
import sys
import os
sys.path+=[os.path.join(os.getcwd(), ".."), os.path.join(os.getcwd(), "../..")] # one and two up

<a name="1-2"></a>
### Notebook Functionality and Appearance
[ToC](#ToC)  
Necessary libraries for notebook functionality:
- A button for hiding/showing the code. By default it is deactivated and can be activated by setting CREATE_BUTTON constant to True. 
> **NOTE: This way, using the function, the button works only in active notebook. If the functionality needs to be preserved in html export, then the code has to be incluced directly into notebook.**
- Set notebook width to 100%.
- Notebook data frame setting for better visibility.
- Initial timestamp setting and logging the start of the execution.

#### Overall Setting Specification

In [2]:
LOGGER_CONFIG_NAME = "logger_file_limit_console"
ADDAPT_WIDTH = True

#### Overall Behaviour Setting

In [3]:
try:
    from src.utils.notebook_support_functions import create_button, get_notebook_name
    NOTEBOOK_NAME = get_notebook_name()
    SUPPORT_FUNCTIONS_READ = True
except:
    NOTEBOOK_NAME = "NO_NAME"
    SUPPORT_FUNCTIONS_READ = False  

In [4]:
from src.utils.logger import Logger
from src.utils.envs import Envs
from src.utils.config import Config
from pandas import options
from IPython.display import display, HTML

In [5]:
options.display.max_rows = 500
options.display.max_columns = 500
envs = Envs()
envs.set_logger(LOGGER_CONFIG_NAME)
Logger().start_timer(f"NOTEBOOK; Notebook name: {NOTEBOOK_NAME}")
if ADDAPT_WIDTH:
    display(HTML("<style>.container { width:100% !important; }</style>")) # notebook width

2024-05-17 11:33:20,726 - file_limit_console - INFO - Logger was created on WS-3000 in branche 001_do_week_01_hw.
2024-05-17 11:33:20,727 - file_limit_console - INFO - Process: NOTEBOOK; Notebook name: W01_HW.ipynb; Timer started;


In [6]:
# create_button()

<a name="1-3"></a>
### External Libraries
[ToC](#ToC)  

In [7]:
from datetime import datetime

from os.path import join

import pandas as pd

import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

from IPython.display import display, HTML

<a name="1-4"></a>
### Internal Code
[ToC](#ToC)  
Code, libraries, classes, functions from within the repository.

In [8]:
from src.utils.date_time_functions import create_datetime_id

<a name="1-5"></a>
### Constants
[ToC](#ToC)  
Constants for the notebook.

> *NOTE: Please use all letters upper.*

#### General Constants
[ToC](#ToC)  

In [9]:
# from src.global_constants import *  # Remember to import only the constants in use
N_ROWS_TO_DISPLAY = 2
FIGURE_SIZE_SETTING = {"autosize": False, "width": 2200, "height": 750}
DATA_PROCESSING_CONFIG_NAME = "data_processing_basic"

#### Constants for Setting Automatic Run
[ToC](#ToC)  

In [10]:
# MANDATORY FOR CONFIG DEFINITION AND NOTEBOOK AND ITS OUTPUTS IDENTIFICATION #########################################
PYTHON_CONFIG_NAME = "python_local"
ID = create_datetime_id(now=datetime.now(), add_micro=False)
# (END) MANDATORY FOR CONFIG DEFINITION AND NOTEBOOK AND ITS OUTPUTS IDENTIFICATION ###################################

#### Python Config Initialisation
[ToC](#ToC)  

In [11]:
envs.set_config(PYTHON_CONFIG_NAME)

#### Notebook Specific Constants
[ToC](#ToC)  

<a name="2"></a>
# ANALYSIS
[ToC](#ToC)  

In [12]:
def print_info_about_data_frame(df: pd.DataFrame) -> None:
    """
    Print overall df stats.
    :param df: DataFrame. Data to have its stats printed.
    """
    print(f"DataFrame type: {str(type(df))}")
    print(f"DataFrame shape: {str(df.shape)}")
    print(f"DataFrame dtypes: {df.dtypes.apply(lambda x: x.name).to_dict()}")
    print("\n")
    print(f"DataFrame head:\n{df.head()}")
    print("\n")
    print(f"DataFrame description:\n{df.describe()}")
    print("\n")

<a name="2-1"></a>
## Data Reading
[ToC](#ToC)  

In [13]:
file_name = "yellow_tripdata_2023-01.parquet"
df_january = pd.read_parquet(join(Config().get_data().path.external_data, file_name))
file_name = "yellow_tripdata_2023-02.parquet"
df_february = pd.read_parquet(join(Config().get_data().path.external_data, file_name))

In [14]:
df_january.shape

(3066766, 19)

In [15]:
df_january.shape

(3066766, 19)

In [16]:
df_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


<a name="2-2"></a>
## Questions
[ToC](#ToC)  

<a name="2-2-1"></a>
### Q1 Read the data for January. How many columns are there?
[ToC](#ToC)  

We'll use the same [NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

16  
17  
18  
19  

In [17]:
answer = f"The number of Columns is: {df_january.shape[1]}"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-2"></a>
### Q2 Computing duration
[ToC](#ToC)  

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

32.59  
42.59  
52.59  
62.59  

In [18]:
ATTR_DURATION = "DURATION"

In [19]:
def add_duration_attribute(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds duration attribute to a data frame.
    """
    df[ATTR_DURATION] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df[ATTR_DURATION] = df[ATTR_DURATION].apply(lambda td: td.total_seconds() / 60)
    
    return df

In [20]:
df_january = add_duration_attribute(df_january)
df_february = add_duration_attribute(df_february)

In [21]:
df_january[[ATTR_DURATION]].head()

,DURATION
0,8.433333
1,6.316667
2,12.750000
3,9.616667
4,10.833333


In [22]:
avg_duration = round(df_january[ATTR_DURATION].std(), 2)
print(avg_duration)

42.59


In [23]:
answer = f"The standard deviation of the trips duration in January is: {avg_duration}"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-3"></a>
### Q3 Dropping outliers
[ToC](#ToC)  

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%  
92%  
95%  
98%  

In [24]:
def remove_outliers(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes outliers from data frame.
    """
    in_between = [1. <= duration <= 60. for duration in df[ATTR_DURATION]]
    
    original_size = df.shape[0]
    df = df.loc[in_between, ]
    
    print(f"DF N of Obs: {original_size}")
    print(f"N of In Between: {sum(in_between)}")
    print(f"New df size    : {df.shape[0]}")
    print(f"Franction: {round(df.shape[0] / original_size, 4)}")
       
    return df, original_size  

In [25]:
df_january, orig_size_january = remove_outliers(df_january)
df_february, orig_size_february = remove_outliers(df_february)

DF N of Obs: 3066766
N of In Between: 3009173
New df size    : 3009173
Franction: 0.9812
DF N of Obs: 2913955
N of In Between: 2855951
New df size    : 2855951
Franction: 0.9801


In [26]:
observations_franction_left = round((df_january.shape[0] + df_february.shape[0]) / (orig_size_january + orig_size_february), 4)
observations_franction_left

0.9807

In [27]:
answer = f"The number of observations left is: {round(observations_franction_left * 100, 2)} %"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-4"></a>
### Q4 One-hot encoding
[ToC](#ToC)  

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

2  
155  
345  
515  
715  

In [28]:
ATTR_PU_LOC_ID = "PULocationID"
ATTR_DO_LOC_ID = "DOLocationID"

ATTRS_FOR_OH = [ATTR_PU_LOC_ID, ATTR_DO_LOC_ID]

In [29]:
df = df_january.copy()

In [30]:
def do_oh_encoding(df: pd.DataFrame) -> pd.DataFrame:
    """
    Does OH encoding using feature vectorizer.
    """
    df = df[ATTRS_FOR_OH].astype(str)
    d = df[ATTRS_FOR_OH].to_dict(orient="records")
    
    return DictVectorizer().fit_transform(d)

In [ ]:
print(f"Januray N of OH Features:{do_oh_encoding(df_january).shape[1]}")
print(f"February N of OH Features:{do_oh_encoding(df_february).shape[1]}")
print(f"Both N of OH Features:{do_oh_encoding(pd.concat((df_january, df_february))).shape[1]}")

Januray N of OH Features:515
February N of OH Features:514


In [ ]:
answer = f"The number of columns for January is: {do_oh_encoding(df_january).shape[1]}"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-5"></a>
### Q5 Training a model
[ToC](#ToC)  

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

3.64  
7.64  
11.64  
16.64  

In [ ]:
Y_train = df_january[ATTR_DURATION].to_numpy()
X_train = do_oh_encoding(df_january)

In [ ]:
print(Y_train.shape)
print(X_train.shape)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, Y_train)

y_hat = lr_model.predict(X_train)

In [ ]:
error = round(mean_squared_error(Y_train, y_hat, squared=False), 4)
error

In [ ]:
answer = f"The RMSE error for the train data is: {error}"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-6"></a>
### Q6 Evaluating the model
[ToC](#ToC) 

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81  
7.81  
11.81  
16.81  

In [ ]:
Y_dev = df_february[ATTR_DURATION].to_numpy()
X_dev = do_oh_encoding(df_february)

In [ ]:
y_hat = lr_model.predict(X_dev)

In [ ]:
error = round(mean_squared_error(Y_dev, y_hat, squared=False), 4)
error

In [ ]:
answer = f"The RMSE error for the dev data is: {error}"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="3"></a>
# Final Timestamp
[ToC](#ToC)  

In [ ]:
Logger().end_timer()